# pmaps_test

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
import time

from invisible_cities.reco.pmap_io           import pmap_writer, S12, S2Si
from invisible_cities.core.system_of_units_c import units
from invisible_cities.reco.pmaps_functions   import read_pmaps, read_run_and_event_from_pmaps_file
from invisible_cities.reco.params            import PMaps

In [ ]:
from invisible_cities.reco.pmaps_functions_c import df_to_pmaps_dict, df_to_s2si_dict

### files

In [ ]:
PMP_file_name  = os.path.join(os.environ['IC_DATA'],'test_pmaps.h5')

### read back tables

In [ ]:
s1df, s2df, s2sidf = read_pmaps(PMP_file_name)
rundf, evtdf = read_run_and_event_from_pmaps_file(PMP_file_name)

In [ ]:
rundf

In [ ]:
rundf.run_number.values

In [ ]:
np.testing.assert_equal(rundf.run_number.values, np.zeros(3, dtype=np.int32))

In [ ]:
evtdf

In [ ]:
evtdf.evt_number.values

### test

In [ ]:
def pmaps():
    """Return  PMAPS."""

    S1 = S12({0:(np.random.rand(5),
         np.random.rand(5))})
    S2 = S12({0:(np.random.rand(5),
         np.random.rand(5))})

    nsipm = {}
    for i in (1,2,3):
        nsipm[i] = np.random.rand(5)
    s2si = {0:nsipm}

    Si = S2Si(s2si)

    # Si = S2Si({0:[(1, np.random.rand(5)),
    #       (2, np.random.rand(5)),
    #       (3, np.random.rand(5))]})

    return PMaps(S1, S2, Si)

In [ ]:

P = pmaps()

run_number = 0
event_info_table = []
for i in range(3):
    event_info_table.append((100+i, int(time.time()) ))

    # write with context
with pmap_writer(PMP_file_name) as write:
    for (event_number, timestamp) in event_info_table:
        write(run_number, event_number, timestamp, P.S1, P.S2, P.S2Si)


In [ ]:
s1df, s2df, s2sidf = read_pmaps(PMP_file_name)
rundf, evtdf = read_run_and_event_from_pmaps_file(PMP_file_name)


In [ ]:
s1df

In [ ]:
s2df

In [ ]:
s2sidf

In [ ]:
S1L = df_to_pmaps_dict(s1df)

In [ ]:
S1L

In [ ]:
S2L = df_to_pmaps_dict(s2df)

In [ ]:
S2L

In [ ]:
S2SiL = df_to_s2si_dict(s2sidf)

In [ ]:
S2SiL

In [ ]:
for s1p in S1L.values():
    for peak in s1p.values():
        print(peak.t, peak.E)
        np.testing.assert_allclose(peak.t, P.S1[0][0])
        np.testing.assert_allclose(peak.E, P.S1[0][1])

In [ ]:
for s2p in S2L.values():
    for peak in s2p.values():
        np.testing.assert_allclose(peak.t, P.S2[0][0])
        np.testing.assert_allclose(peak.E, P.S2[0][1])


In [ ]:
for s in S2SiL.values():
    for peak in s.values():
        for i, E in peak.items():
            np.testing.assert_allclose(E, P.S2Si[0][i])